In [2]:
import pandas as pd
import numpy as np
import nltk
import lemminflect

import spacy


In [3]:
headers = pd.read_csv("decomp_ud_trees.csv", header=None, nrows=2,
                      index_col=0, keep_default_na=False, na_filter=False).values.tolist()
roles_wide = pd.read_csv("decomp_ud_trees.csv", header=[0, 1], index_col=0)
roles_wide.columns = pd.MultiIndex.from_arrays(headers)

# convert from json to spacy parse
nlp = spacy.load("en_core_web_sm")
def from_json(parse_string):
    j = eval(parse_string)
    return spacy.tokens.Doc(nlp.vocab).from_json(j)

roles_wide["Arg.Parse"] = roles_wide["Arg.Parse"].apply(from_json)
roles_wide["spacy_parse"] = roles_wide["spacy_parse"].apply(from_json)

print(roles_wide.shape)
roles_wide.head()


(7499, 45)


Sentence.ID Pred.Lemma  Arg.Phrase Pred.Token Arg.Tokens.Begin  \
                                                                             
0    en-ud-dev.conllu 1       come  this story          3                4   
1    en-ud-dev.conllu 1       come  this story          3                4   
2  en-ud-dev.conllu 100        cry     America          1                0   
3  en-ud-dev.conllu 100        cry     America          1                0   
4  en-ud-dev.conllu 100        cry        wolf          1                2   

  Arg.Tokens.End Gram.Func Split Sent.Grammatical Annotator.ID  ... Response  \
                                                                ... volition   
0              5     nsubj   dev              5.0            7  ...      1.0   
1              5     nsubj   dev              5.0           20  ...      3.0   
2              0     nsubj   dev              2.0           46  ...      5.0   
3              0     nsubj   dev              5.0           15  ...      5.0   
4              2      dobj   dev              5.0           26  ...      3.0   

                                                                         tree  \
  was_for_benefit was_used                                                      
0             2.0      1.0  (SINV\n  (PP-1 (IN From) (NP (DT the) (NNP AP)...   
1             3.0      5.0  (SINV\n  (PP-1 (IN From) (NP (DT the) (NNP AP)...   
2             5.0      5.0  (S\n  (S\n    (NP-SBJ+NNP America)\n    (VP (V...   
3             3.0      5.0  (S\n  (S\n    (NP-SBJ+NNP America)\n    (VP (V...   
4             3.0      3.0  (S\n  (S\n    (NP-SBJ+NNP America)\n    (VP (V...   

                                            sentence  \
                                                       
0                     From the AP comes this story :   
1                     From the AP comes this story :   
2  America cried wolf in Iraq , and what 's scary...   
3  America cried wolf in Iraq , and what 's scary...   
4  America cried wolf in Iraq , and what 's scary...   

                                         spacy_parse  \
                                                       
0             (From, the, AP, comes, this, story, :)   
1             (From, the, AP, comes, this, story, :)   
2  (America, cried, wolf, in, Iraq, ,, and, what,...   
3  (America, cried, wolf, in, Iraq, ,, and, what,...   
4  (America, cried, wolf, in, Iraq, ,, and, what,...   

                        Arg.Tree Arg.treepos Predicate      Arg.Parse  
                                                                       
0  (NP-SBJ (DT this) (NN story))        (2,)     comes  (this, story)  
1  (NP-SBJ (DT this) (NN story))        (2,)     comes  (this, story)  
2           (NP-SBJ+NNP America)      (0, 0)     cried      (America)  
3           (NP-SBJ+NNP America)      (0, 0)     cried      (America)  
4                   (NP+NN wolf)   (0, 1, 1)     cried         (wolf)  

[5 rows x 45 columns]

In [ ]:
sentences = roles_wide[
    ["Sentence.ID", "Pred.Lemma", "Gram.Func", "Arg.treepos", "Split",
      "Arg.Parse", "tree", "Pred.Token"]
    ].drop_duplicates(
        subset=[("Sentence.ID", ""), ("Arg.treepos", "")]
    ).pivot_table(index=["Sentence.ID", "Pred.Lemma", "tree", "Pred.Token"],
                  columns="Gram.Func",
                  values=["Arg.treepos", "Split", "Arg.Parse"],
                  aggfunc=list).reset_index()
sentences['tree'] = sentences['tree'].apply(
    nltk.tree.ParentedTree.fromstring
)
sentences.columns = sentences.columns.droplevel(1)

sentences


/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_23595/99592803.py:15: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  ).pivot_table(index=["Sentence.ID", "Pred.Lemma", "tree", "Pred.Token"],


Sentence.ID Pred.Lemma  \
Gram.Func                                       
0               en-ud-dev.conllu 1       come   
1             en-ud-dev.conllu 100        cry   
2            en-ud-dev.conllu 1002       call   
3             en-ud-dev.conllu 101        run   
4            en-ud-dev.conllu 1013     resume   
...                            ...        ...   
2740       en-ud-train.conllu 9947    explain   
2741       en-ud-train.conllu 9961       have   
2742       en-ud-train.conllu 9962       make   
2743       en-ud-train.conllu 9973       have   
2744       en-ud-train.conllu 9995      phone   

                                                        tree Pred.Token  \
Gram.Func                                                                 
0          [[[From], [(DT the), (NNP AP)]], [[comes]], [[...          3   
1          [[[America], [(VBD cried), (NP+NN wolf), (PP-L...          1   
2          [[[Abbas], [,], [(NP (NP (DT a) (NN refugee)) ...         16   
3          [[I], [[ran], [(IN across), (NP (DT this) (NN ...          1   
4          [[[In], [(NP+NN response), (PP (IN to) (NP (DT...          7   
...                                                      ...        ...   
2740       [[Staff], [[(VBD explained), (NP (NN insurance...          1   
2741       [[They], [[have], [(NP (DT a) (JJ great) (NN s...          1   
2742       [[[The], [employees]], [[(VBP make), (S\n  (NP...          2   
2743       [[They], [[have], [(DT a), (NML (NN credit) (N...          1   
2744       [[[I], [(VBD phoned), (NP (DT this) (NN compan...          1   

                                                   Arg.Parse       \
Gram.Func                                               dobj iobj   
0                                                        NaN  NaN   
1                                                   [(wolf)]  NaN   
2                                    [(the, refugee, issue)]  NaN   
3                                                        NaN  NaN   
4          [(its, policy, of, targeting, militant, leader...  NaN   
...                                                      ...  ...   
2740                               [(insurance, procedures)]  NaN   
2741       [(a, great, selection, of, wine, from, all, ov...  NaN   
2742                                                 [(you)]  NaN   
2743                            [(a, credit, card, minimum)]  NaN   
2744                                       [(this, company)]  NaN   

                                                                        \
Gram.Func                                              nsubj nsubjpass   
0                                            [(this, story)]       NaN   
1                                                [(America)]       NaN   
2          [(Abbas, ,, a, refugee, himself, from, what, i...       NaN   
3                                                      [(I)]       NaN   
4                                                 [(Israel)]       NaN   
...                                                      ...       ...   
2740                                               [(Staff)]       NaN   
2741                                                [(They)]       NaN   
2742                                      [(The, employees)]       NaN   
2743                                                [(They)]       NaN   
2744                                                   [(I)]       NaN   

              Arg.treepos                             Split                \
Gram.Func            dobj iobj     nsubj nsubjpass     dobj iobj    nsubj   
0                     NaN  NaN    [(2,)]       NaN      NaN  NaN    [dev]   
1             [(0, 1, 1)]  NaN  [(0, 0)]       NaN    [dev]  NaN    [dev]   
2                [(2, 1)]  NaN    [(0,)]       NaN    [dev]  NaN    [dev]   
3                     NaN  NaN    [(0,)]       NaN      NaN  NaN    [dev]   
4                [(3, 1)]  NaN    [(2,)]       NaN    [dev]  NaN    [dev]   
...      

In [5]:
# can only passify sentences with both nsubj AND dobj
# cannot be already passive (contain nsubj)

# remove sentences containing multiple predicates, no subject, or no object
sentences = sentences.dropna(
    subset=[('Arg.treepos','dobj'), ('Arg.treepos','nsubj')]).drop_duplicates(
        subset=[("Sentence.ID", "")], keep=False)

# remove sentences with more than one object
sentences[('Arg.treepos','dobj')] = sentences[('Arg.treepos','dobj')].apply(
    lambda x: x if len(x) == 1 else np.nan)
sentences = sentences.dropna(subset=[('Arg.treepos','dobj')]).drop(
    columns=[("Arg.Parse", "nsubjpass"), ("Arg.treepos", "nsubjpass"), ("Split", "nsubjpass")]
)

sentences


Sentence.ID Pred.Lemma  \
Gram.Func                                       
1             en-ud-dev.conllu 100        cry   
2            en-ud-dev.conllu 1002       call   
4            en-ud-dev.conllu 1013     resume   
10           en-ud-dev.conllu 1041       mean   
13           en-ud-dev.conllu 1064       have   
...                            ...        ...   
2740       en-ud-train.conllu 9947    explain   
2741       en-ud-train.conllu 9961       have   
2742       en-ud-train.conllu 9962       make   
2743       en-ud-train.conllu 9973       have   
2744       en-ud-train.conllu 9995      phone   

                                                        tree Pred.Token  \
Gram.Func                                                                 
1          [[[America], [(VBD cried), (NP+NN wolf), (PP-L...          1   
2          [[[Abbas], [,], [(NP (NP (DT a) (NN refugee)) ...         16   
4          [[[In], [(NP+NN response), (PP (IN to) (NP (DT...          7   
10         [[[you], [(VBP mean), (NP (NP+NNP miramar) (NP...          1   
13         [[they], [[have], [(NP (PRP$ their) (JJ own) (...          1   
...                                                      ...        ...   
2740       [[Staff], [[(VBD explained), (NP (NN insurance...          1   
2741       [[They], [[have], [(NP (DT a) (JJ great) (NN s...          1   
2742       [[[The], [employees]], [[(VBP make), (S\n  (NP...          2   
2743       [[They], [[have], [(DT a), (NML (NN credit) (N...          1   
2744       [[[I], [(VBD phoned), (NP (DT this) (NN compan...          1   

                                                   Arg.Parse       \
Gram.Func                                               dobj iobj   
1                                                   [(wolf)]  NaN   
2                                    [(the, refugee, issue)]  NaN   
4          [(its, policy, of, targeting, militant, leader...  NaN   
10                                      [(miramar, florida)]  NaN   
13         [(their, own, website, which, you, can, easily...  NaN   
...                                                      ...  ...   
2740                               [(insurance, procedures)]  NaN   
2741       [(a, great, selection, of, wine, from, all, ov...  NaN   
2742                                                 [(you)]  NaN   
2743                            [(a, credit, card, minimum)]  NaN   
2744                                       [(this, company)]  NaN   

                                                                 Arg.treepos  \
Gram.Func                                              nsubj            dobj   
1                                                [(America)]     [(0, 1, 1)]   
2          [(Abbas, ,, a, refugee, himself, from, what, i...        [(2, 1)]   
4                                                 [(Israel)]        [(3, 1)]   
10                                                   [(you)]     [(0, 1, 1)]   
13                                                  [(they)]        [(1, 1)]   
...                                                      ...             ...   
2740                                               [(Staff)]     [(1, 0, 1)]   
2741                                                [(They)]        [(1, 1)]   
2742                                      [(The, employees)]  [(1, 0, 1, 0)]   
2743                                                [(They)]        [(1, 1)]   
2744                                                   [(I)]     [(0, 1, 1)]   

                            Split                
Gram.Func iobj     nsubj     dobj iobj    nsubj  
1          NaN  [(0, 0)]    [dev]  NaN    [dev]  
2          NaN    [(0,)]    [dev]  NaN    [dev]  
4          NaN    [(2,)]    [dev]  NaN    [dev]  
10         NaN  [(0, 0)]    [dev]  NaN    [dev]  
13         NaN    [(0,)]    [dev]  NaN    [dev]  
...        ...       ...      ...  ...      ...  
2740       NaN    [(0,)]  [train]  NaN  [train]  
2741       NaN    [(

In [ ]:
def swap_subj_obj(tree, positions):
    positions = [eval(p) for p in positions]
    subtrees = []
    new_positions = []

    for position in positions:
        subtree = tree[position]
        if isinstance(subtree, str):
            # position = position[:-1]
            subtree = tree[position[:-1]]
        subtrees.append(subtree.copy())
        new_positions.append(position)

    # swap subtrees in-place
    subtrees = subtrees[::-1]
    for subtree, tp in zip(subtrees, new_positions):
        tree[tp] = subtree
        
    return subtrees, new_positions


In [7]:
import requests
import bs4


def wiki_is_plural(root):
    url = 'https://en.wiktionary.org/api/rest_v1/page/definition/'
    headers = {
         "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"
        }
    
    wiki = requests.get(url + root, headers=headers).json()
    if 'en' not in wiki:
        return False
    for result in wiki['en']:
        for definition in result["definitions"]:            
            soup = bs4.BeautifulSoup(definition["definition"], parse_only=bs4.SoupStrainer('a'))
            for link in soup.find_all('a', href=True):
                if link['href'] == "/wiki/Appendix:Glossary#plural_number":
                    return True
    return False


def is_plural(subtree, spacy_root):
    # no conjunctions
    for child in subtree:
        if isinstance(child, nltk.tree.ParentedTree) and "CC" in child.label():
            return True

    root = spacy_root.text
    # get index in subtree
    tp = subtree.leaf_treeposition(subtree.leaves().index(root))
    if subtree[tp[:-1]].label().split("-")[-1] in ["NNS", "NNPS"]:
        return True
    if subtree[tp[:-1]].label().split("-")[-1] in ["NN", "NNP"]:
        return False

    # make request to wikitionary API
    if wiki_is_plural(root):
        return True
            
    return False


In [8]:
prn_inflect = {
    "i": "me",
    "me": "I",
    "he": "him",
    "him": "he",
    "she": "her",
    "her": "she",
    "we": "us",
    "us": "we",
    "they": "them",
    "them": "they"
}

anaphors = {
    "itself", "themself", "themselves", "herself", "himself", "myself", "yourself"
}

def fix_pronouns(subtrees, spacy_roots):
    # replace prps in the subtree
    for subtree, root in zip(subtrees, spacy_roots):
        root = root.text
        if root.lower() in prn_inflect:         
            idx = subtree.leaves().index(root)
            tp = subtree.leaf_treeposition(idx)
            subtree[tp] = prn_inflect[root.lower()]


def fix_cases(tree, subj, obj):
    for subtree in [subj, obj]:
        tp = subtree.leaf_treeposition(0)
        arg_first_word = subtree[tp]
        
        # check if subj is first word in sentence
        if subtree == subj and arg_first_word == tree.leaves()[0]:
            # make upper case
            arg_first_word = arg_first_word[0].upper() + arg_first_word[1:]
            subtree[tp] = arg_first_word

        # else only capitalize proper nouns
        elif subtree[tp[:-1]].label().split("-")[-1] not in ["NNP", "NNPS"]:
            # make lower lower case
            arg_first_word = arg_first_word[0].lower() + arg_first_word[1:]
            subtree[tp] = arg_first_word


In [9]:
# imposed -> was imposed by
# imposes -> is imposed by
# will impose -> will be imposed by

# Future tense verbs have already been removed from the dataset

def in_conjunction(subtree):
    if isinstance(subtree, str):
        return False
    
    for child in subtree:
        if isinstance(child, nltk.tree.ParentedTree) and "CC" in child.label():
            return True

    if "VP" in subtree.label():
        # do not go any higher up than VP
        return False
    elif subtree.parent():
        return in_conjunction(subtree.parent())
    return False


def make_verb_passive(tree, vpos, new_subj, subj_root):
    verb = tree[vpos]
    
    tenses = pattern.en.tenses(verb)
    lemma = lemminflect.getLemma(verb, upos="VERB")[0]
    past_participle = lemminflect.getInflection(lemma, tag='VBN')[0]
    plural = is_plural(new_subj, subj_root)

    # move the verb's particle, if any
    lp = tree.leaves().index(verb)
    if lp + 1 < len(tree.leaves()):
        right_pos =  tree.leaf_treeposition(lp + 1)
        right_sibling = tree[right_pos[:-1]]
        if "PRT" in right_sibling.label():
            past_participle += " " + " ".join(right_sibling.leaves())
            # remove the sibling
            tree[right_pos] = ''
    
    # move any adverbs
    # "[adv] were [past participle] by" -> "were [adv] [past participle] by"
    if lp > 0:
        left_pos =  tree.leaf_treeposition(lp - 1)
        left_sibling = tree[left_pos[:-1]]
        if "RB" in left_sibling.label() and "ADVP" in left_sibling.parent().label():
            past_participle = " ".join(left_sibling.leaves()) + " " + past_participle
            # remove the sibling
            tree[left_pos] = ''

    if pattern.en.PAST in tenses:
        if plural:
            new_verb = f"were {past_participle} by"
        else:
            new_verb = f"was {past_participle} by"
    else:
        if plural:
            new_verb = f"are {past_participle} by"
        else:
            new_verb = f"is {past_participle} by"
    tree[vpos] = new_verb


def make_passive(row, inplace=False, return_args=True):
    if inplace:
        tree = row[("tree", "")]
    else:
        tree = row[("tree", "")].copy(deep=True)
    vpos = tree.leaf_treeposition(row[("Pred.Token", "")])

    # cannot passify when there is more than one verb
    if in_conjunction(tree[vpos[:-1]]):
        print(tree.leaves())
        print(tree[vpos[:-1]])
        return np.nan

    # former obj becomes new subj
    (new_subj, new_obj), arg_treepos = swap_subj_obj(
        tree, [row[("Arg.treepos", "nsubj")][0], row[("Arg.treepos", "dobj")][0]])
    
    # use spacy dependency parsing to get root of each arg
    subj_root = row[("Arg.Parse", "dobj")][0][0].sent.root
    obj_root = row[("Arg.Parse", "nsubj")][0][0].sent.root

    # remove sentences with unbound anaphors
    # TODO: also remove pronouns that were previously bound using coreference
    # https://github.com/huggingface/neuralcoref?tab=readme-ov-file
    if subj_root.text in anaphors:
        return np.nan
    
    # correct arguments and verb in place
    make_verb_passive(tree, vpos, new_subj, subj_root)
    fix_pronouns([new_subj, new_obj], [subj_root, obj_root])
    fix_cases(tree, new_subj, new_obj)

    if return_args:
        return (
            tree, 
            vpos,
            arg_treepos[0],
            arg_treepos[1]
            )
    else:
        return tree

def make_passive_(row):
    try:
        return make_passive(row)
    except:
        print(row)
        return [np.nan] * 4


In [ ]:
sentences[["passive tree", "verb_pos", "subj_pos", "obj_pos"]] = sentences.apply(
    make_passive_,
    axis=1,
    result_type="expand")
sentences = sentences.dropna(subset=[("passive tree", '')]).reset_index(drop=True)


In [11]:
sentences.rename(columns={"tree": "active tree"}, inplace=True)
sentences.drop(columns=['Arg.Parse', 'Pred.Token'], inplace=True)

sentences["active sentence"] = sentences["active tree"].apply(
    lambda x: " ".join(x.leaves())
)
sentences["passive sentence"] = sentences["passive tree"].apply(
    lambda x: " ".join(x.leaves())
)

sentences["active verb"] = sentences.apply(
    lambda x: x[("active tree", '')][x[("verb_pos", '')]],
    axis=1
)
sentences["active nsubj"] = sentences.apply(
    lambda x: " ".join(x[("active tree", '')][x[("subj_pos", '')]].leaves()),
    axis=1
)
sentences["active dobj"] = sentences.apply(
    lambda x: " ".join(x[("active tree", '')][x[("obj_pos", '')]].leaves()),
    axis=1
)

sentences["passive verb"] = sentences.apply(
    lambda x: x[("passive tree", '')][x[("verb_pos", '')]],
    axis=1
)
sentences["passive nsubj"] = sentences.apply(
    lambda x: " ".join(x[("passive tree", '')][x[("subj_pos", '')]].leaves()),
    axis=1
)
sentences["passive dobj"] = sentences.apply(
    lambda x: " ".join(x[("passive tree", '')][x[("obj_pos", '')]].leaves()),
    axis=1
)

sentences


/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_23595/2947687723.py:2: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  sentences.drop(columns=['Arg.Parse', 'Pred.Token'], inplace=True)


Sentence.ID Pred.Lemma  \
Gram.Func                                       
0            en-ud-dev.conllu 1002       call   
1            en-ud-dev.conllu 1013     resume   
2            en-ud-dev.conllu 1041       mean   
3            en-ud-dev.conllu 1064       have   
4            en-ud-dev.conllu 1096       bulk   
..                             ...        ...   
970        en-ud-train.conllu 9947    explain   
971        en-ud-train.conllu 9961       have   
972        en-ud-train.conllu 9962       make   
973        en-ud-train.conllu 9973       have   
974        en-ud-train.conllu 9995      phone   

                                                 active tree     Arg.treepos  \
Gram.Func                                                               dobj   
0          [[[Abbas], [,], [(NP (NP (DT a) (NN refugee)) ...        [(2, 1)]   
1          [[[In], [(NP+NN response), (PP (IN to) (NP (DT...        [(3, 1)]   
2          [[[you], [(VBP mean), (NP (NP+NNP miramar) (NP...     [(0, 1, 1)]   
3          [[they], [[have], [(NP (PRP$ their) (JJ own) (...        [(1, 1)]   
4          [[They], [[bulk], [up], [(ADJP (RB too) (JJ mu...        [(1, 2)]   
..                                                       ...             ...   
970        [[Staff], [[(VBD explained), (NP (NN insurance...     [(1, 0, 1)]   
971        [[They], [[have], [(NP (DT a) (JJ great) (NN s...        [(1, 1)]   
972        [[[The], [employees]], [[(VBP make), (S\n  (NP...  [(1, 0, 1, 0)]   
973        [[They], [[have], [(DT a), (NML (NN credit) (N...        [(1, 1)]   
974        [[[I], [(VBD phoned), (NP (DT this) (NN compan...     [(0, 1, 1)]   

                            Split                \
Gram.Func iobj     nsubj     dobj iobj    nsubj   
0          NaN    [(0,)]    [dev]  NaN    [dev]   
1          NaN    [(2,)]    [dev]  NaN    [dev]   
2          NaN  [(0, 0)]    [dev]  NaN    [dev]   
3          NaN    [(0,)]    [dev]  NaN    [dev]   
4          NaN    [(0,)]    [dev]  NaN    [dev]   
..         ...       ...      ...  ...      ...   
970        NaN    [(0,)]  [train]  NaN  [train]   
971        NaN    [(0,)]  [train]  NaN  [train]   
972        NaN    [(0,)]  [train]  NaN  [train]   
973        NaN    [(0,)]  [train]  NaN  [train]   
974        NaN  [(0, 0)]  [train]  NaN  [train]   

                                                passive tree  ... subj_pos  \
Gram.Func                                                     ...            
0          [[[The], [refugee], [issue]], [,], [[was calle...  ...     (0,)   
1          [[[In], [(NP+NN response), (PP (IN to) (NP (DT...  ...     (2,)   
2          [[[(NP+NNP Miramar), (NP-LOC+NNP florida)], [(...  ...   (0, 0)   
3          [[[(PRP$ Their), (JJ own), (NN website)], [(WH...  ...     (0,)   
4          [[[(RB Too), (JJ much)], [space]], [[is bulked...  ...     (0,)   
..                                                       ...  ...      ...   
970        [[[Insurance], [procedures]], [[(VBD were expl...  ...     (0,)   
971        [[[(DT A), (JJ great), (NN selection)], [(IN o...  ...     (0,)   
972        [[You], [[(VBP is made by), (S\n  (NP-SBJ (DT ...  ...     (0,)   
973        [[[A], [(NN credit), (NN card)], [minimum]], [...  ...     (0,)   
974        [[[(DT This), (NN company)], [(VBD was phoned ...  ...   (0, 0)   

                obj_pos                                    active sentence  \
Gram.Func                                                                    
0                (2, 1)  Abbas , a refugee himself from what is today t...   
1                (3, 1)  In response to the rockets , Israel resumed it...   
2             (0, 1, 1)  you mean miramar florida theyy have good seafo...   
3                (1, 1)  they have their own website which you can easi...   
4                (1, 2)                        They bulk up too much space   
..                  ...                                                ...   
970           (1, 0, 1)  Staff explained 

In [12]:
def treepos_to_index(tree, argpos, arg, sentence, pos="subj"):
    leaves = tree.leaves()
    arg_leaves = arg.split(" ")

    # predicate always has token length of 1 in the tree, even when passive
    if pos == "verb":
        arg_leaves = [arg]
        arg_tree_length = 1
    else:
        arg_tree_length = len(arg_leaves)

    matches = []
    potential_matches = []
    for i in range(len(leaves) - arg_tree_length + 1):
        if leaves[i : i + arg_tree_length] == arg_leaves:
            potential_matches.append((i, i + arg_tree_length))
            treepos = tree.leaf_treeposition(i)
            if treepos[:len(argpos)] == argpos:
                matches.append((i, i + arg_tree_length))
    assert len(matches) == 1, (argpos, leaves, arg_leaves, potential_matches)

    if pos == "obj":
        # after the predicate, so we add 2 to index b/c the verb was added to
        diff = len(sentence.split(" ")) - len(leaves)
        match = (matches[0][0] + diff, matches[0][1] + diff)
    elif pos == "verb":
        # add 2 to the end of the matched sequence
        diff = len(sentence.split(" ")) - len(leaves)
        match = (matches[0][0], matches[0][1] + diff)
        arg_leaves = arg.split(" ")
    else:
        match = matches[0]
    assert sentence.split(" ")[match[0]: match[1]] == arg_leaves, (sentence.split(" ")[match[0]: match[1]], arg_leaves)
    return match


In [13]:
sentences.columns


MultiIndex([(     'Sentence.ID',      ''),
            (      'Pred.Lemma',      ''),
            (     'active tree',      ''),
            (     'Arg.treepos',  'dobj'),
            (     'Arg.treepos',  'iobj'),
            (     'Arg.treepos', 'nsubj'),
            (           'Split',  'dobj'),
            (           'Split',  'iobj'),
            (           'Split', 'nsubj'),
            (    'passive tree',      ''),
            (        'verb_pos',      ''),
            (        'subj_pos',      ''),
            (         'obj_pos',      ''),
            ( 'active sentence',      ''),
            ('passive sentence',      ''),
            (     'active verb',      ''),
            (    'active nsubj',      ''),
            (     'active dobj',      ''),
            (    'passive verb',      ''),
            (   'passive nsubj',      ''),
            (    'passive dobj',      '')],
           names=[None, 'Gram.Func'])

In [14]:
treepos_to_index(
    sentences[("passive tree", "")][0], sentences[("verb_pos", "")][0],
    sentences[("passive verb", "")][0], sentences[("passive sentence", "")][0],
    pos="verb"
)


(4, 7)

In [15]:
sentences["passive nsubj idx"] = sentences.apply(
    lambda x: treepos_to_index(
        x[("passive tree", "")], x[("subj_pos", "")], x[("passive nsubj", "")],
        x[("passive sentence", "")], pos="subj"),
    axis=1
)
sentences["passive dobj idx"] = sentences.apply(
    lambda x: treepos_to_index(
        x[("passive tree", "")], x[("obj_pos", "")], x[("passive dobj", "")],
        x[("passive sentence", "")], pos="obj"),
    axis=1
)

sentences["passive verb idx"] = sentences.apply(
    lambda x: treepos_to_index(
        x[("passive tree", "")], x[("verb_pos", "")], x[("passive verb", "")],
        x[("passive sentence", "")], pos="verb"),
    axis=1
)


In [16]:
old_sentences = sentences.copy(deep=True)


In [17]:
sentences.columns


MultiIndex([(      'Sentence.ID',      ''),
            (       'Pred.Lemma',      ''),
            (      'active tree',      ''),
            (      'Arg.treepos',  'dobj'),
            (      'Arg.treepos',  'iobj'),
            (      'Arg.treepos', 'nsubj'),
            (            'Split',  'dobj'),
            (            'Split',  'iobj'),
            (            'Split', 'nsubj'),
            (     'passive tree',      ''),
            (         'verb_pos',      ''),
            (         'subj_pos',      ''),
            (          'obj_pos',      ''),
            (  'active sentence',      ''),
            ( 'passive sentence',      ''),
            (      'active verb',      ''),
            (     'active nsubj',      ''),
            (      'active dobj',      ''),
            (     'passive verb',      ''),
            (    'passive nsubj',      ''),
            (     'passive dobj',      ''),
            ('passive nsubj idx',      ''),
            ( 'passive dobj idx'

In [21]:
sentences = old_sentences.copy(deep=True)


In [18]:
sentences.drop(columns=[("Split", "iobj"), ("Arg.treepos", "iobj"), 'active tree',
                         'passive tree', 'verb_pos', 'subj_pos', 'obj_pos'
                         ], inplace=True)
sentences[("Split", "dobj")] = sentences[("Split", "dobj")].apply(lambda x: x[0])
sentences[("Split", "nsubj")] = sentences[("Split", "nsubj")].apply(lambda x: x[0])
value_vars = [
    ("Split", "dobj"), ("Split", "nsubj"),
    ]
sentences = sentences.reset_index().melt(
    value_vars = value_vars, value_name="_split_",
    id_vars=[x for x in sentences.columns if x not in value_vars]
    ).drop(columns=[None]).rename(columns={"_split_": "Split"})

sentences.columns = map(lambda x: 
                        " ".join(x).strip() if not isinstance(x, str) else x, sentences.columns)
sentences = sentences.drop(
    columns=["Gram.Func", "Split", 'Arg.treepos dobj', 'Arg.treepos nsubj']
    ).set_index(['Sentence.ID','Pred.Lemma'])
sentences.head()
sentences.to_csv("decomp_ud_passive_sentences.csv")


/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_23595/918832764.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  sentences.drop(columns=[("Split", "iobj"), ("Arg.treepos", "iobj"), 'active tree',


In [19]:
sentences.columns


Index(['active sentence', 'passive sentence', 'active verb', 'active nsubj',
       'active dobj', 'passive verb', 'passive nsubj', 'passive dobj',
       'passive nsubj idx', 'passive dobj idx', 'passive verb idx'],
      dtype='object')

In [36]:
headers = pd.read_csv("decomp_ud_modified_sentences.csv", header=None, nrows=2,
                      index_col=0, keep_default_na=False).values.tolist()
judgements = pd.read_csv("decomp_ud_modified_sentences.csv", header=[0, 1], index_col=0)
judgements.columns = pd.MultiIndex.from_arrays(headers)

proto_roles_names = [x[1] for x in judgements.columns[10:24].tolist()]
judgements = judgements.set_index([x for x in judgements.columns if x[1] == '']).reset_index()
# new_index = [x[0] for x in judgements.columns if x[1] == '' and x[0] not in [
#     "sentence", "Predicate", "Pred.Token", "Arg.Tokens.Begin", "Arg.Tokens.End"
# ]]
new_index = ["Sentence.ID", "Pred.Lemma", "Arg.Phrase", "Arg.Stripped", "Gram.Func", "Split", "Roleset", "Arg"]
print(judgements.shape)
judgements.head()


(7499, 48)


Sentence.ID Pred.Lemma  Arg.Phrase Pred.Token Arg.Tokens.Begin  \
                                                                             
0    en-ud-dev.conllu 1       come  this story          3                4   
1    en-ud-dev.conllu 1       come  this story          3                4   
2  en-ud-dev.conllu 100        cry     America          1                0   
3  en-ud-dev.conllu 100        cry     America          1                0   
4  en-ud-dev.conllu 100        cry        wolf          1                2   

  Arg.Tokens.End Gram.Func Split Sent.Grammatical Annotator.ID  ...  \
                                                                ...   
0              5     nsubj   dev              5.0            7  ...   
1              5     nsubj   dev              5.0           20  ...   
2              0     nsubj   dev              2.0           46  ...   
3              0     nsubj   dev              5.0           15  ...   
4              2      dobj   dev              5.0           26  ...   

                    Response                                              \
  change_of_state_continuous existed_after existed_before existed_during   
0                        1.0           4.0            2.0            4.0   
1                        1.0           5.0            5.0            5.0   
2                        5.0           5.0            5.0            5.0   
3                        5.0           5.0            5.0            5.0   
4                        1.0           3.0            3.0            3.0   

                                                                    
  instigation partitive sentient volition was_for_benefit was_used  
0         2.0       2.0      2.0      1.0             2.0      1.0  
1         5.0       2.0      1.0      3.0             3.0      5.0  
2         5.0       5.0      5.0      5.0             5.0      5.0  
3         5.0       5.0      5.0      5.0             3.0      5.0  
4         3.0       3.0      1.0      3.0             3.0      3.0  

[5 rows x 48 columns]

In [28]:
sentences.head()


,,active sentence,passive sentence,active verb,active nsubj,active dobj,passive verb,passive nsubj,passive dobj,passive nsubj idx,passive dobj idx,passive verb idx
Sentence.ID,Pred.Lemma,,,,,,,,,,,
en-ud-dev.conllu 1002,call,"Abbas , a refugee himself from what is today t...","The refugee issue , was called by abbas , a re...",called,"Abbas , a refugee himself from what is today t...",the refugee issue,was called by,The refugee issue,"abbas , a refugee himself from what is today t...","(0, 3)","(7, 22)","(4, 7)"
en-ud-dev.conllu 1013,resume,"In response to the rockets , Israel resumed it...","In response to the rockets , its policy of tar...",resumed,Israel,its policy of targeting militant leaders in ai...,was resumed by,its policy of targeting militant leaders in ai...,israel,"(6, 15)","(18, 19)","(15, 18)"
en-ud-dev.conllu 1041,mean,you mean miramar florida theyy have good seafo...,Miramar florida is meant by you theyy have goo...,mean,you,miramar florida,is meant by,Miramar florida,you,"(0, 2)","(5, 6)","(2, 5)"
en-ud-dev.conllu 1064,have,they have their own website which you can easi...,Their own website which you can easily find us...,have,they,their own website which you can easily find us...,is had by,Their own website which you can easily find us...,them,"(0, 12)","(15, 16)","(12, 15)"
en-ud-dev.conllu 1096,bulk,They bulk up too much space,Too much space is bulked up by them,bulk,They,too much space,is bulked up by,Too much space,them,"(0, 3)","(8, 9)","(3, 7)"


In [26]:
sentences.columns


Index(['active sentence', 'passive sentence', 'active verb', 'active nsubj',
       'active dobj', 'passive verb', 'passive nsubj', 'passive dobj',
       'passive nsubj idx', 'passive dobj idx', 'passive verb idx'],
      dtype='object')

In [ ]:
responses_df = judgements["Response"]
responses_df[["Arg.Tokens.Begin", "Annotator.ID"]] = judgements[["Arg.Tokens.Begin", "Annotator.ID"]]
responses_df[new_index] = judgements[new_index]

responses_df = responses_df.set_index(["Sentence.ID", "Pred.Lemma"])
responses_df.head()
passive_responses = pd.merge(sentences, responses_df,
                             left_index=True, right_index=True,
                             how="inner").reset_index()
print(passive_responses.shape)
passive_responses.head()

# applicable_df = judgements["Applicable"]
# applicable_df[new_index] = judgements[new_index]
# applicable_df = applicable_df.set_index(["Sentence.ID", "Pred.Lemma"])
# applicable_df.head()
# passive_applicable = pd.merge(sentences, applicable_df, left_index=True, right_index=True).reset_index()


(7820, 35)


/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_23595/900345552.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  responses_df[["Arg.Tokens.Begin", "Annotator.ID"]] = judgements[["Arg.Tokens.Begin", "Annotator.ID"]]
/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_23595/900345552.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  responses_df[["Arg.Tokens.Begin", "Annotator.ID"]] = judgements[["Arg.Tokens.Begin", "Annotator.ID"]]
/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipyke

,Sentence.ID,Pred.Lemma,active sentence,passive sentence,active verb,active nsubj,active dobj,passive verb,passive nsubj,passive dobj,...,was_for_benefit,was_used,Arg.Tokens.Begin,Annotator.ID,Arg.Phrase,Arg.Stripped,Gram.Func,Split,Roleset,Arg
0,en-ud-dev.conllu 1002,call,"Abbas , a refugee himself from what is today t...","The refugee issue , was called by abbas , a re...",called,"Abbas , a refugee himself from what is today t...",the refugee issue,was called by,The refugee issue,"abbas , a refugee himself from what is today t...",...,4.0,5.0,0,26,"Abbas, a refugee himself from what is today th...",Abbas,nsubj,dev,call.01,1
1,en-ud-dev.conllu 1002,call,"Abbas , a refugee himself from what is today t...","The refugee issue , was called by abbas , a re...",called,"Abbas , a refugee himself from what is today t...",the refugee issue,was called by,The refugee issue,"abbas , a refugee himself from what is today t...",...,3.0,4.0,0,76,"Abbas, a refugee himself from what is today th...",Abbas,nsubj,dev,call.01,1
2,en-ud-dev.conllu 1002,call,"Abbas , a refugee himself from what is today t...","The refugee issue , was called by abbas , a re...",called,"Abbas , a refugee himself from what is today t...",the refugee issue,was called by,The refugee issue,"abbas , a refugee himself from what is today t...",...,5.0,1.0,17,26,the refugee issue,the refugee issue,dobj,dev,call.01,NaN
3,en-ud-dev.conllu 1002,call,"Abbas , a refugee himself from what is today t...","The refugee issue , was called by abbas , a re...",called,"Abbas , a refugee himself from what is today t...",the refugee issue,was called by,The refugee issue,"abbas , a refugee himself from what is today t...",...,5.0,5.0,17,93,the refugee issue,the refugee issue,dobj,dev,call.01,NaN
4,en-ud-dev.conllu 1013,resume,"In response to the rockets , Israel resumed it...","In response to the rockets , its policy of tar...",resumed,Israel,its policy of targeting militant leaders in ai...,was resumed by,its policy of targeting militant leaders in ai...,israel,...,5.0,5.0,6,43,Israel,Israel,nsubj,dev,resume.01,NaN


In [69]:
responses_df.to_csv("active_responses.csv")


In [70]:
responses_df = judgements["Response"]
responses_df[["Arg.Tokens.Begin", "Annotator.ID"]] = judgements[["Arg.Tokens.Begin", "Annotator.ID"]]
responses_df[new_index] = judgements[new_index]

responses_df.duplicated(
    subset=["Sentence.ID", "Pred.Lemma", "Arg.Tokens.Begin", "Annotator.ID"]
).sum()


/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_23595/4056884167.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  responses_df[["Arg.Tokens.Begin", "Annotator.ID"]] = judgements[["Arg.Tokens.Begin", "Annotator.ID"]]
/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipykernel_23595/4056884167.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  responses_df[["Arg.Tokens.Begin", "Annotator.ID"]] = judgements[["Arg.Tokens.Begin", "Annotator.ID"]]
/var/folders/lq/kd3wh6952vg99n5ppf4srz9w0000gn/T/ipy

16

In [ ]:
passive_responses[passive_responses.duplicated(
    subset=["Sentence.ID", "Pred.Lemma", "Arg.Tokens.Begin", "Annotator.ID"]
)]


,Sentence.ID,Pred.Lemma,active sentence,passive sentence,active verb,active nsubj,active dobj,passive verb,passive nsubj,passive dobj,...,was_for_benefit,was_used,Arg.Tokens.Begin,Annotator.ID,Arg.Phrase,Arg.Stripped,Gram.Func,Split,Roleset,Arg
19,en-ud-dev.conllu 1096,bulk,They bulk up too much space,Too much space is bulked up by them,bulk,They,too much space,is bulked up by,Too much space,them,...,2.0,3.0,3,20,too much space,space,dobj,dev,bulk.01,1
222,en-ud-dev.conllu 1923,offer,"They offer a good portions at a great price , ...",A good portions are offered by them at a great...,offer,They,a good portions,are offered by,A good portions,them,...,5.0,1.0,0,87,They,They,nsubj,dev,offer.01,0
816,en-ud-test.conllu 465,love,WE AT HOME LOVE IT AT $ 80 +++,IT is LOVED by us AT HOME AT $ 80 +++,LOVE,WE AT HOME,IT,is LOVED by,IT,us AT HOME,...,5.0,5.0,0,43,WE AT HOME,WE,nsubj,test,love.01,0
1141,en-ud-train.conllu 10628,know,They know their job and you do not have to wat...,Their job is known by them and you do not have...,know,They,their job,is known by,Their job,them,...,4.0,3.0,0,20,They,They,nsubj,train,know.01,0
1705,en-ud-train.conllu 12076,know,SHE KNOWS GREAT FOOD AND DINING EXPERIENCES .,GREAT FOOD AND DINING EXPERIENCES is KNOWN by ...,KNOWS,SHE,GREAT FOOD AND DINING EXPERIENCES,is KNOWN by,GREAT FOOD AND DINING EXPERIENCES,her,...,5.0,5.0,2,26,GREAT FOOD AND DINING EXPERIENCES,FOOD AND DINING EXPERIENCES,dobj,train,know.01,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7815,en-ud-train.conllu 9973,have,They have a credit card minimum .,A credit card minimum is had by them .,have,They,a credit card minimum,is had by,A credit card minimum,them,...,4.5,5.0,0,26,They,They,nsubj,train,have.03,0
7816,en-ud-train.conllu 9995,phone,I phoned this company for advice on our office...,This company was phoned by me for advice on ou...,phoned,I,this company,was phoned by,This company,me,...,5.0,5.0,0,7,I,I,nsubj,train,phone.01,0
7817,en-ud-train.conllu 9995,phone,I phoned this company for advice on our office...,This company was phoned by me for advice on ou...,phoned,I,this company,was phoned by,This company,me,...,5.0,5.0,0,81,I,I,nsubj,train,phone.01,0
7818,en-ud-train.conllu 9995,phone,I phoned this company for advice on our office...,This company was phoned by me for advice on ou...,phoned,I,this company,was phoned by,This company,me,...,3.0,5.0,2,26,this company,this company,dobj,train,phone.01,1


In [77]:
passive_responses = passive_responses.drop_duplicates(
    # subset=["Sentence.ID", "Pred.Lemma", "Arg.Tokens.Begin", "Annotator.ID"]
)
print(passive_responses.shape)
passive_responses.head()


(3910, 35)


,Sentence.ID,Pred.Lemma,active sentence,passive sentence,active verb,active nsubj,active dobj,passive verb,passive nsubj,passive dobj,...,was_for_benefit,was_used,Arg.Tokens.Begin,Annotator.ID,Arg.Phrase,Arg.Stripped,Gram.Func,Split,Roleset,Arg
0,en-ud-dev.conllu 1002,call,"Abbas , a refugee himself from what is today t...","The refugee issue , was called by abbas , a re...",called,"Abbas , a refugee himself from what is today t...",the refugee issue,was called by,The refugee issue,"abbas , a refugee himself from what is today t...",...,4.0,5.0,0,26,"Abbas, a refugee himself from what is today th...",Abbas,nsubj,dev,call.01,1
1,en-ud-dev.conllu 1002,call,"Abbas , a refugee himself from what is today t...","The refugee issue , was called by abbas , a re...",called,"Abbas , a refugee himself from what is today t...",the refugee issue,was called by,The refugee issue,"abbas , a refugee himself from what is today t...",...,3.0,4.0,0,76,"Abbas, a refugee himself from what is today th...",Abbas,nsubj,dev,call.01,1
2,en-ud-dev.conllu 1002,call,"Abbas , a refugee himself from what is today t...","The refugee issue , was called by abbas , a re...",called,"Abbas , a refugee himself from what is today t...",the refugee issue,was called by,The refugee issue,"abbas , a refugee himself from what is today t...",...,5.0,1.0,17,26,the refugee issue,the refugee issue,dobj,dev,call.01,NaN
3,en-ud-dev.conllu 1002,call,"Abbas , a refugee himself from what is today t...","The refugee issue , was called by abbas , a re...",called,"Abbas , a refugee himself from what is today t...",the refugee issue,was called by,The refugee issue,"abbas , a refugee himself from what is today t...",...,5.0,5.0,17,93,the refugee issue,the refugee issue,dobj,dev,call.01,NaN
4,en-ud-dev.conllu 1013,resume,"In response to the rockets , Israel resumed it...","In response to the rockets , its policy of tar...",resumed,Israel,its policy of targeting militant leaders in ai...,was resumed by,its policy of targeting militant leaders in ai...,israel,...,5.0,5.0,6,43,Israel,Israel,nsubj,dev,resume.01,NaN


In [79]:

passive_responses.to_csv("decomp_ud_passive_responses.csv")
print(passive_responses.shape)
passive_responses.head()


(3910, 35)


,Sentence.ID,Pred.Lemma,active sentence,passive sentence,active verb,active nsubj,active dobj,passive verb,passive nsubj,passive dobj,...,was_for_benefit,was_used,Arg.Tokens.Begin,Annotator.ID,Arg.Phrase,Arg.Stripped,Gram.Func,Split,Roleset,Arg
0,en-ud-dev.conllu 1002,call,"Abbas , a refugee himself from what is today t...","The refugee issue , was called by abbas , a re...",called,"Abbas , a refugee himself from what is today t...",the refugee issue,was called by,The refugee issue,"abbas , a refugee himself from what is today t...",...,4.0,5.0,0,26,"Abbas, a refugee himself from what is today th...",Abbas,nsubj,dev,call.01,1
1,en-ud-dev.conllu 1002,call,"Abbas , a refugee himself from what is today t...","The refugee issue , was called by abbas , a re...",called,"Abbas , a refugee himself from what is today t...",the refugee issue,was called by,The refugee issue,"abbas , a refugee himself from what is today t...",...,3.0,4.0,0,76,"Abbas, a refugee himself from what is today th...",Abbas,nsubj,dev,call.01,1
2,en-ud-dev.conllu 1002,call,"Abbas , a refugee himself from what is today t...","The refugee issue , was called by abbas , a re...",called,"Abbas , a refugee himself from what is today t...",the refugee issue,was called by,The refugee issue,"abbas , a refugee himself from what is today t...",...,5.0,1.0,17,26,the refugee issue,the refugee issue,dobj,dev,call.01,NaN
3,en-ud-dev.conllu 1002,call,"Abbas , a refugee himself from what is today t...","The refugee issue , was called by abbas , a re...",called,"Abbas , a refugee himself from what is today t...",the refugee issue,was called by,The refugee issue,"abbas , a refugee himself from what is today t...",...,5.0,5.0,17,93,the refugee issue,the refugee issue,dobj,dev,call.01,NaN
4,en-ud-dev.conllu 1013,resume,"In response to the rockets , Israel resumed it...","In response to the rockets , its policy of tar...",resumed,Israel,its policy of targeting militant leaders in ai...,was resumed by,its policy of targeting militant leaders in ai...,israel,...,5.0,5.0,6,43,Israel,Israel,nsubj,dev,resume.01,NaN
